In [4]:
import optuna

In [5]:
import flappy_bird_gymnasium
import gymnasium
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import randint

In [6]:
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image
import time
import os
import model

cuda


In [7]:
def create_agent(env,BUFFER_SIZE = int(1e5),BATCH_SIZE = 64,GAMMA = 0.99,TAU = 1e-3,UPDATE_EVERY=4):
        # if gpu is to be used
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    class QNetwork(nn.Module):
        def __init__(self, state_size, action_size, seed):
            """
            Initialize parameters and build model.
            Params:
            - state_size (int): Dimension of each state
            - action_size (int): Dimension of each action
            - seed (int): Random seed
            - fc1_unit (int): Number of nodes in first hidden layer
            - fc2_unit (int): Number of nodes in second hidden layer
            """
            super(QNetwork, self).__init__() ## calls __init__ method of nn.Module class
            self.seed = torch.manual_seed(seed)

            
            fc1_unit=128
            fc2_unit=128
            self.fc1=nn.Linear(state_size,fc1_unit)
            self.fc2=nn.Linear(fc1_unit,fc2_unit)
            self.fc3=nn.Linear(fc2_unit,action_size)
            

        def forward(self, state):
            """
            Build a network that maps state -> action values.
            """
            x=F.relu(self.fc1(state))
            x=F.relu(self.fc2(x))
            return self.fc3(x)
            
    class Agent():
        """
        Interacts with and learns form environment.
        """

        def __init__(self, state_size, action_size, seed):
            """
            Initialize an Agent object.
            Params:
            - state_size (int): dimension of each state
            - action_size (int): dimension of each action
            - seed (int): random seed
            """

            self.state_size = state_size
            self.action_size = action_size
            self.seed = random.seed(seed)

            # Q-Network
            self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
            self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)

            self.optimizer = optim.Adam(self.qnetwork_local.parameters(),lr=0.0001)

            # Replay Memory
            self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)

            # Initialize time step (for updating every UPDATE_EVERY steps)
            self.t_step = 0

        def step(self, state, action, reward, next_step, done):
            # Save experience in replay memory
            self.memory.add(state, action, reward, next_step, done)

            # Learn every UPDATE_EVERY time steps.
            self.t_step = (self.t_step+1) % UPDATE_EVERY
            if self.t_step == 0:
                # If enough samples are available in memory, get random subset and learn
                if len(self.memory) > BATCH_SIZE:
                    experience = self.memory.sample()
                    self.learn(experience, GAMMA)


        def act(self, state, eps = 0):
            """
            Returns action for given state as per current policy.
            Params:
            - state (array_like): current state
            - eps (float): epsilon, for epsilon-greedy action selection
            """
            state = torch.from_numpy(state).float().unsqueeze(0).to(device)
            self.qnetwork_local.eval()
            with torch.no_grad():
                action_values = self.qnetwork_local(state)
            self.qnetwork_local.train()

            # Epsilon-greedy action selction
            if random.random() > eps:
                return np.argmax(action_values.cpu().data.numpy())
            else:
                return random.choice(np.arange(self.action_size))

        def learn(self, experiences, gamma):
            """
            Update value parameters using given batch of experience tuples.
            Params:
            - experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples
            - gamma (float): discount factor
            """
            states, actions, rewards, next_states, dones = experiences

            
            ## TODO: compute and minimize the loss
            # Get max predicted Q values (for next states) from target model
            Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)

            # Compute Q targets for current states
            Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

            # Get expected Q values from local model
            Q_expected = self.qnetwork_local(states).gather(1, actions)

            # Compute loss
            loss = F.mse_loss(Q_expected, Q_targets)

            # Minimize the loss
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            

            # ------------------- update target network ------------------- #
            self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)

        def soft_update(self, local_model, target_model, tau):
            """
            Soft update model parameters.
            θ_target = τ*θ_local + (1 - τ)*θ_target

            Params:
            - local model (PyTorch model): weights will be copied from
            - target model (PyTorch model): weights will be copied to
            - tau (float): interpolation parameter
            """
            for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
                target_param.data.copy_(tau*local_param.data + (1-tau)*target_param.data)


    class ReplayBuffer:
        """
        Fixed-size buffe to store experience tuples.
        """
        def __init__(self, action_size, buffer_size, batch_size, seed):
            """
            Initialize a ReplayBuffer object.
            Params:
            - action_size (int): dimension of each action
            - buffer_size (int): maximum size of buffer
            - batch_size (int): size of each training batch
            - seed (int): random seed
            """
            self.action_size = action_size
            self.memory = deque(maxlen=buffer_size)
            self.batch_size = batch_size
            self.experiences = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

            self.seed = random.seed(seed)

        def add(self,state, action, reward, next_state,done):
            """
            Add a new experience to memory.
            """
            e = self.experiences(state,action,reward,next_state,done)
            self.memory.append(e)

        def sample(self):
            """
            Randomly sample a batch of experiences from memory.
            """
            experiences = random.sample(self.memory, k=self.batch_size)

            states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
            actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
            rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
            next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
            dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

            return (states,actions,rewards,next_states,dones)

        def __len__(self):
            """
            Return the current size of internal memory.
            """
            return len(self.memory)
    return Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.n, seed=0)

In [8]:
def objective(trial):
    env = gymnasium.make("FlappyBird-v0")

    # set up matplotlib
    is_ipython = 'inline' in matplotlib.get_backend()
    if is_ipython:
        from IPython import display

    plt.ion()

    # if gpu is to be used
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    #Parameter
    BUFFER_SIZE = trial.suggest_int("BUFFER_SIZE", 1e4, 1e6)
    BATCH_SIZE = trial.suggest_int("BATCH_SIZE", 32, 256)
    GAMMA = trial.suggest_float("GAMMA", 0.8, 0.999)
    TAU = trial.suggest_float("TAU", 1e-4, 1e-2)
    UPDATE_EVERY = trial.suggest_int("UPDATE_EVERY", 1, 10)

    

    # Init agent
    agent = create_agent(env,BUFFER_SIZE,BATCH_SIZE,GAMMA,TAU,UPDATE_EVERY)
    checkpoint_path = "checkpoints/model_checkpoint"
    checkpoint_dir = "checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)  

    def DQN(n_episodes=1000, eps_start=0.9, eps_end=0.01, eps_decay=0.995):
        """
        Deep Q-Learning Training
        """
        scores = []  
        scores_window = deque(maxlen=100)  #100 scores
        eps = eps_start
        best_score = 1


        for i_episode in range(n_episodes):
            state, _ = env.reset()
            score = 0
            done = False

            while not done:
                action = agent.act(state, eps)
                next_state, reward, done, _, _ = env.step(action)
                agent.step(state, action, reward, next_state, done)

                state = next_state
                score += reward

            scores.append(score)
            scores_window.append(score)
            eps = max(eps * eps_decay, eps_end)  #Decrease eps

            # Save highest checkpoint
            score_avg = np.mean(scores_window)
            if score_avg > best_score:
                best_score = score_avg
                torch.save(agent.qnetwork_local.state_dict(), f"{checkpoint_path}_best_{score_avg:.2f}.pth")
                print(f" New best model saved! Score: {score_avg:.2f}")

            # Save checkpoint every 1000 eps
            if i_episode % 500 == 0:
                torch.save(agent.qnetwork_local.state_dict(), f"{checkpoint_path}_{i_episode}_{score_avg:.2f}.pth")
                print(f"Checkpoint saved at {time.strftime('%Y-%m-%d %H:%M:%S')}")

            print(f'\rEpisode {i_episode}\tAverage Score: {score_avg:.2f}', end="")

            # Early stop
            if score_avg >= 100:
                print(f'\n✅ Environment solved in {i_episode} episodes! Avg Score: {score_avg:.2f}')
                torch.save(agent.qnetwork_local.state_dict(), 'final_model.pth')
                break

        env.close()  
        return np.mean(scores_window)
    return DQN()

In [ ]:
study = optuna.create_study(direction="maximize",study_name="DQN_trials", storage="sqlite:///DQN_trials.db'")
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

[I 2025-03-03 07:45:57,063] A new study created in memory with name: no-name-91474039-6fe6-4ae2-aee8-4eedccc72cc3


cuda
cuda


/users/eleves-a/2022/felix.ndereyimana/.local/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/users/eleves-a/2022/felix.ndereyimana/.local/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


Checkpoint saved at 2025-03-03 07:46:00
Episode 499	Average Score: -1.40Checkpoint saved at 2025-03-03 07:46:40
Episode 996	Average Score: -1.39

[I 2025-03-03 07:47:25,071] Trial 0 finished with value: -1.5309999999999997 and parameters: {'BUFFER_SIZE': 146575, 'BATCH_SIZE': 42, 'GAMMA': 0.9378342300559974, 'TAU': 0.003043388258881088, 'UPDATE_EVERY': 5}. Best is trial 0 with value: -1.5309999999999997.


...

Episode 999	Average Score: 2.12cuda
cuda
Checkpoint saved at 2025-03-03 11:05:45
Episode 499	Average Score: -2.36Checkpoint saved at 2025-03-03 11:06:29
Episode 916	Average Score: 0.90 New best model saved! Score: 1.02
Episode 917	Average Score: 1.02 New best model saved! Score: 1.06
Episode 920	Average Score: 1.01 New best model saved! Score: 1.09
Episode 922	Average Score: 1.09 New best model saved! Score: 1.13
Episode 927	Average Score: 1.11 New best model saved! Score: 1.18
Episode 929	Average Score: 1.18 New best model saved! Score: 1.21
Episode 930	Average Score: 1.21 New best model saved! Score: 1.24
Episode 931	Average Score: 1.24 New best model saved! Score: 1.25
Episode 932	Average Score: 1.25 New best model saved! Score: 1.28
Episode 933	Average Score: 1.28 New best model saved! Score: 1.29
Episode 940	Average Score: 1.27 New best model saved! Score: 1.33
Episode 941	Average Score: 1.33 New best model saved! Score: 1.38
Episode 942	Average Score: 1.38 New best model saved! S

[I 2025-03-03 11:07:16,466] Trial 98 finished with value: 1.7899999999999994 and parameters: {'BUFFER_SIZE': 742233, 'BATCH_SIZE': 118, 'GAMMA': 0.8657794121446323, 'TAU': 0.004321547052141618, 'UPDATE_EVERY': 4}. Best is trial 61 with value: 5.441999999999986.


Episode 999	Average Score: 1.79cuda
cuda
Checkpoint saved at 2025-03-03 11:07:16
Episode 499	Average Score: -1.30Checkpoint saved at 2025-03-03 11:08:34
Episode 691	Average Score: 0.971 New best model saved! Score: 1.00
Episode 692	Average Score: 1.00 New best model saved! Score: 1.02
Episode 693	Average Score: 1.02 New best model saved! Score: 1.04
Episode 694	Average Score: 1.04 New best model saved! Score: 1.05
Episode 695	Average Score: 1.05 New best model saved! Score: 1.07
Episode 696	Average Score: 1.07 New best model saved! Score: 1.11
Episode 697	Average Score: 1.11 New best model saved! Score: 1.13
Episode 698	Average Score: 1.13 New best model saved! Score: 1.14
Episode 700	Average Score: 1.13 New best model saved! Score: 1.17
Episode 702	Average Score: 1.17 New best model saved! Score: 1.18
Episode 703	Average Score: 1.18 New best model saved! Score: 1.19
Episode 705	Average Score: 1.18 New best model saved! Score: 1.24
Episode 706	Average Score: 1.24 New best model saved! 

[I 2025-03-03 11:10:03,247] Trial 99 finished with value: 1.955999999999999 and parameters: {'BUFFER_SIZE': 481379, 'BATCH_SIZE': 101, 'GAMMA': 0.8936556851531223, 'TAU': 0.006086884306415569, 'UPDATE_EVERY': 1}. Best is trial 61 with value: 5.441999999999986.


Episode 999	Average Score: 1.96Best hyperparameters:  {'BUFFER_SIZE': 757624, 'BATCH_SIZE': 228, 'GAMMA': 0.8777525034865641, 'TAU': 0.005566248249524419, 'UPDATE_EVERY': 3}


In [9]:
print(study.best_params)

{'BUFFER_SIZE': 757624, 'BATCH_SIZE': 228, 'GAMMA': 0.8777525034865641, 'TAU': 0.005566248249524419, 'UPDATE_EVERY': 3}
